<a href="https://colab.research.google.com/github/chernoskutov1989dv/Self-Education/blob/main/15_3_rnn128_64_se_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding  # Embedding отсекает ненужные связи(нули), оптимизируя обьем памяти
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

In [ ]:
open ("/content/train_data_true.txt")

<_io.TextIOWrapper name='/content/train_data_true.txt' mode='r' encoding='UTF-8'>

In [ ]:

with open('train_data_true.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')  # убираем первый невидимый символ

In [ ]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('думайте', 1), ('позитивно', 4), ('и', 29), ('верьте', 3), ('в', 23), ('свою', 3), ('способность', 1), ('достигать', 1), ('отличных', 1), ('результатов', 1)]


In [ ]:
data = tokenizer.texts_to_sequences([texts])
#res = to_categorical(data[0], num_classes=maxWordsCount)
#print(res.shape)

res = np.array( data[0] )

inp_words = 3
n = res.shape[0] - inp_words

X = np.array([res[i:i + inp_words] for i in range(n)])  # теперь это набор чисел!!
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

In [ ]:
model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))  # Embedding слой должен быть первым!! 
model.add(SimpleRNN(128, activation='tanh', return_sequences=True))  # 1ый рекуррентный слой в формате MANY TO MANY!
model.add(SimpleRNN(64, activation='tanh'))  # 2ой рекуррентный слой many to one
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 256)            256000    
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 3, 128)            49280     
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 64)                12352     
                                                                 
 dense_1 (Dense)             (None, 1000)              65000     
                                                                 
Total params: 382,632
Trainable params: 382,632
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
31/31 [==============================] - 2s 8ms/step - loss: 6.8746 - accuracy: 0.0264
Epoch 2/50
31/31 [==============================] - 0s 8ms/step - loss: 6.2264 - accuracy: 0.0396
Epoch 3/50
31/31 [==============================] - 0s 7ms/step - loss: 5.8118 - accuracy: 0.0426
Epoch 4/50
31/31 [==============================] - 0s 7ms/step - loss: 5.5177 - accuracy: 0.0862
Epoch 5/50
31/31 [==============================] - 0s 7ms/step - loss: 5.1703 - accuracy: 0.1034
Epoch 6/50
31/31 [==============================] - 0s 7ms/step - loss: 4.8105 - accuracy: 0.1359
Epoch 7/50
31/31 [==============================] - 0s 7ms/step - loss: 4.4582 - accuracy: 0.2160
Epoch 8/50
31/31 [==============================] - 0s 7ms/step - loss: 4.1197 - accuracy: 0.2972
Epoch 9/50
31/31 [==============================] - 0s 8ms/step - loss: 3.7895 - accuracy: 0.3874
Epoch 10/50
31/31 [==============================] - 0s 7ms/step - loss: 3.4819 - accuracy: 0.4696
Epoch 11/50
31/31 [

In [ ]:
def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = data[i: i + inp_words] # теперь берем конкретные индексы, а не ОНЕ векторы!!
        inp = np.expand_dims(x, axis =0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res


res = buildPhrase("Окружите себя позитивными")
print(res)

1/1 [==============================] - 0s 15ms/step
Окружите себя позитивными людьми которые верят в ваши мечты поддерживают ваши идеи оставайся позитивным прости других инвестируй в себя доверяй своим инстинктам не
